In [ ]:
import kaggle
import os
import zipfile

In [ ]:
data_dir = "./Data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

Using here an initial dataset found at https://www.kaggle.com/datasets/paulchambaz/google-street-view/data

This will be replaced in the future

In [ ]:
!kaggle datasets download -d paulchambaz/google-street-view -p {data_dir}

In [ ]:
with zipfile.ZipFile(f"{data_dir}/google-street-view.zip", 'r') as zip_ref:
    zip_ref.extractall(data_dir)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
df = pd.read_csv('./Data/dataset/coords.csv', names=['latitude', 'longitude'])
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))


Adding country labels for each datapoint. 

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf = gdf.set_crs(world.crs)
intersect_gdf = gpd.sjoin(gdf, world, how="left", op='intersects')
df['country'] = intersect_gdf['name']

Dropping countries not in the list at ./Data/country_list.csv (i.e. only keeping European countries)

In [ ]:
country_list = pd.read_csv('./Data/country_list.csv')
european = country_list['country'].unique()
europe_df = df[df['country'].isin(european)]
print(europe_df.head())
print(europe_df.shape)

Index column corresponds to image number in the data.

In [ ]:
europe_df.to_csv('./Data/dataset/european_images.csv', index=True)